<h1>Project 1 Part 2

The following code was used for Part 2 of Project 1 for CS4111 Introduction to Database Systems.<br>
<b>Copyright 2020, Xilin Wang, All rights reserved.

In [32]:
import requests
import json
from random import randint
from datetime import datetime
from bs4 import BeautifulSoup

<h3>Webscraping and parsing part.

In [33]:
#use this function to get a list of place id based on query input
def get_place_id(query,api_key,radius=15):
    
    #URL Set up
    base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
    api_url = base_url+"query="
    url = api_url+query+"&key="+api_key
    if radius:
        url+="&radius="+str(radius)
        
    #The list to be returned
    output_list = list()
    
    
    import requests
    import json
    
    response = requests.get(url) # get initial response
        
    while(True):
        data = response.json()
        # get place_id and append to list
        for place in data['results']: 
            place_id = place['place_id']
            output_list.append(place_id)
        
        # Now determine if next page exists, if doesn't exist, jump out of the loop
        try: 
            token = data['next_page_token']
        except:
            break
        
        # if exist, make a new url based on next_page_token
        newurl = base_url + 'pagetoken=' + token + '&key=' + api_key
        
        # get data from next page, if not available, wait a bit and try again
        response = requests.get(newurl) 
        while response.json().get('status') != 'OK':
            from time import sleep
            from random import random 
            sleep(random())
            response = requests.get(newurl)     
    return output_list

In [34]:
with open('/Users/xilinwang/Documents/credentials/google_place_api','r') as f:
    api_key = f.readline().strip()

In [35]:
base = "https://maps.googleapis.com/maps/api/place/details/json?place_id="

In [151]:
restaurants_query = "Japanese restaurants near Columbia University"
pid_list = get_place_id(restaurants_query,api_key,radius=1500)

In [152]:
result = list()

# loop through the pid list and get information on each restaurant 
for pid in pid_list:
    url = base+pid+"&key="+api_key
    response = requests.get(url)
    data = response.json()
    place = data['result']
    adr = place['adr_address']
    soup = BeautifulSoup(adr)
    stnum = place['address_components'][0]['short_name']
    stname = place['address_components'][1]['short_name']
    city = soup.find('span', {'class':"locality"}).get_text()
    state = soup.find('span', {'class':"region"}).get_text()
    postal = soup.find('span', {'class':"postal-code"}).get_text()
    loc = {'stnum':stnum,'stname':stname,'city':city,'state':state,'postal':postal}
    rname = place['name']
    try:
        p_lvl = place['price_level']
    except:
        p_lvl = 2 # default to 2, arbitrary
    reviews = list()
    try: 
        for review in place['reviews']:
            uid = review['author_url'][36:57]
            name = review['author_name']
            rating = review['rating'] 
            ts = review['time']
            time = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            detail = review['text']
            rando = str(randint(0, 9))
            urid = uid + rando
            reviews.append({'uid':uid, 'urid':urid, 'uname':name,'rating':rating, 'time':time, 'detail':detail})
    except:
        reviews = None
    try:
        website = place['website']
    except:
        website = place['url']
    menu_link = website
    mid = 1
    phone = place['formatted_phone_number']
    
    #changee this to the category used for searching
    cat = 'Japanese'
    
    #generating realistic data for the purpose of the project
    if int(phone[13])%2 == 0:
        otype =['take-out','delivery','dine-in']
    elif int(phone[13])%3 == 0:
        otype =['take-out','dine-in']
    else:
        otype =['take-out','delivery']
    
    #dietary need is also generated
    if int(phone[12])%2 == 0:
        dname = ['Vegan', 'Vegetarian']
    elif int(phone[12])%3 == 0:
        dname = ['Vegan', 'Vegetarian', 'Halal']
    else:
        dname = ['Vegetarian', 'Gluten Free']
        
    result.append({'rid':pid, 'rname':rname, 'web':website, 'phone':phone, 
                   'cat':cat, 'otype':otype, 'rev':reviews, 'dname':dname, 
                   'mid':mid, 'mlink':menu_link, 'price':p_lvl, 'loc':loc})

<h3>SQL query part

Below are codes we used to generate SQL insertion queries. We had to use this method to insert data into our PostgreSQL database since we haven't learned an automated way to do this.

In [138]:
def insert_restaurant(data):
    base_insert = 'INSERT INTO Restaurant VALUES'
    for place in data:
        value = (place['rid'],place['rname'],place['web'],place['phone'],place['cat'])
        print(base_insert, value,';')

In [153]:
insert_restaurant(result)

INSERT INTO Restaurant VALUES ('ChIJu3iXyGr2wokR29wWAqLfn8Q', 'Nikko', 'http://nikkohibachigrill.com/', '(212) 531-1188', 'Japanese') ;
INSERT INTO Restaurant VALUES ('ChIJUZGzuBL3wokRslOX3yc9LCk', 'Kikoo Sushi - Columbia', 'http://www.kikoowest.com/', '(212) 533-2211', 'Japanese') ;
INSERT INTO Restaurant VALUES ('ChIJSQm0BGr2wokRMlxPSzeSl2g', 'Jin Ramen West Harlem', 'https://125.jinramen.com/', '(646) 559-2862', 'Japanese') ;
INSERT INTO Restaurant VALUES ('ChIJD86ws4JYwokRz3jXJAWgGdM', 'Kouzan', 'https://qmenu.us/#/kouzan-new-york', '(212) 280-8099', 'Japanese') ;
INSERT INTO Restaurant VALUES ('ChIJjYXMA6r3wokRqsATKxPO-Vg', 'Yuzu Premium Sushi Restaurant', 'http://www.yuzunewyork.com/', '(646) 861-3883', 'Japanese') ;
INSERT INTO Restaurant VALUES ('ChIJVVUpOG_2wokRFUtuo_v80Ts', 'Sushi Sushi', 'https://www.sushisushinyc.com/', '(212) 866-7876', 'Japanese') ;
INSERT INTO Restaurant VALUES ('ChIJFalicmT2wokR02HksZFX9VA', 'Geisha Sushi', 'http://www.geisha3470ny.com/', '(212) 862-780

In [82]:
def insert_users(data):
    base_insert = 'INSERT INTO Users VALUES'
    for place in data:
        rev = place['rev']
        if rev == None:
            continue
        else:
            for review in rev:
                uid = review['uid']
                name = review['uname']
                value = (uid,name)
                print(base_insert,value,';')

In [154]:
insert_users(result)

INSERT INTO Users VALUES ('107325724009413713612', 'Sonny Decker') ;
INSERT INTO Users VALUES ('111130985700064660030', 'AL “L.A” Salcedo') ;
INSERT INTO Users VALUES ('111676253614334282381', 'Gordon Crane') ;
INSERT INTO Users VALUES ('116127320157848663247', 'Henry Gao') ;
INSERT INTO Users VALUES ('100611160727382240523', 'Xavier Savage') ;
INSERT INTO Users VALUES ('113940049200647362048', 'Donald J Barnes') ;
INSERT INTO Users VALUES ('113350429450063061606', 'Mark Hollinger') ;
INSERT INTO Users VALUES ('118202851192349639887', 'Kadiatou Kamara') ;
INSERT INTO Users VALUES ('108045925603217899608', 'Abdelazim Wahdan') ;
INSERT INTO Users VALUES ('111594382664159330327', 'B3D Music') ;
INSERT INTO Users VALUES ('100956114396258860220', 'Lipika Halder') ;
INSERT INTO Users VALUES ('114415284558194219537', 'Liz Carcagno') ;
INSERT INTO Users VALUES ('100159621215660693526', 'Arjun Soto') ;
INSERT INTO Users VALUES ('108720298012115524138', 'Finnley Houston') ;
INSERT INTO Users VAL

In [42]:
def insert_has(data):
    base_insert = 'INSERT INTO Has VALUES'
    for place in data:
        rev = place['rev']
        if rev == None:
            continue
        else:
            for review in rev:
                uid = review['uid']
                if int(uid[17])%5 == 0:
                    need = ['Vegetarian']
                elif int(uid[17]) == 7:
                    need = ['Vegan','Gluten Free']
                elif int(uid[17]) == 3:
                    need = ['Halal','Vegan']
                else:
                    need = None
                
                if need == None:
                    continue
                else:
                    for dn in need:
                        value = (uid, dn)
                        print(base_insert,value,';')

In [155]:
insert_has(result)

INSERT INTO Has VALUES ('107325724009413713612', 'Halal') ;
INSERT INTO Has VALUES ('107325724009413713612', 'Vegan') ;
INSERT INTO Has VALUES ('111130985700064660030', 'Vegetarian') ;
INSERT INTO Has VALUES ('116127320157848663247', 'Halal') ;
INSERT INTO Has VALUES ('116127320157848663247', 'Vegan') ;
INSERT INTO Has VALUES ('100611160727382240523', 'Vegetarian') ;
INSERT INTO Has VALUES ('111594382664159330327', 'Vegetarian') ;
INSERT INTO Has VALUES ('100956114396258860220', 'Vegetarian') ;
INSERT INTO Has VALUES ('100159621215660693526', 'Halal') ;
INSERT INTO Has VALUES ('100159621215660693526', 'Vegan') ;
INSERT INTO Has VALUES ('100724510814613837975', 'Vegan') ;
INSERT INTO Has VALUES ('100724510814613837975', 'Gluten Free') ;
INSERT INTO Has VALUES ('111088452330769280255', 'Vegetarian') ;
INSERT INTO Has VALUES ('118193037536942680649', 'Vegetarian') ;
INSERT INTO Has VALUES ('106307021902158047692', 'Vegan') ;
INSERT INTO Has VALUES ('106307021902158047692', 'Gluten Free') 

In [44]:
def insert_offers(data):
    base_insert = 'INSERT INTO offers VALUES'
    for place in data:
        for otype in place['otype']:
            value = (place['rid'],otype)
            print(base_insert,value,';')

In [156]:
insert_offers(result)

INSERT INTO offers VALUES ('ChIJu3iXyGr2wokR29wWAqLfn8Q', 'take-out') ;
INSERT INTO offers VALUES ('ChIJu3iXyGr2wokR29wWAqLfn8Q', 'delivery') ;
INSERT INTO offers VALUES ('ChIJu3iXyGr2wokR29wWAqLfn8Q', 'dine-in') ;
INSERT INTO offers VALUES ('ChIJUZGzuBL3wokRslOX3yc9LCk', 'take-out') ;
INSERT INTO offers VALUES ('ChIJUZGzuBL3wokRslOX3yc9LCk', 'delivery') ;
INSERT INTO offers VALUES ('ChIJSQm0BGr2wokRMlxPSzeSl2g', 'take-out') ;
INSERT INTO offers VALUES ('ChIJSQm0BGr2wokRMlxPSzeSl2g', 'delivery') ;
INSERT INTO offers VALUES ('ChIJSQm0BGr2wokRMlxPSzeSl2g', 'dine-in') ;
INSERT INTO offers VALUES ('ChIJD86ws4JYwokRz3jXJAWgGdM', 'take-out') ;
INSERT INTO offers VALUES ('ChIJD86ws4JYwokRz3jXJAWgGdM', 'dine-in') ;
INSERT INTO offers VALUES ('ChIJjYXMA6r3wokRqsATKxPO-Vg', 'take-out') ;
INSERT INTO offers VALUES ('ChIJjYXMA6r3wokRqsATKxPO-Vg', 'dine-in') ;
INSERT INTO offers VALUES ('ChIJVVUpOG_2wokRFUtuo_v80Ts', 'take-out') ;
INSERT INTO offers VALUES ('ChIJVVUpOG_2wokRFUtuo_v80Ts', 'delivery'

In [145]:
def insert_is_at_location(data):
    base_insert = 'INSERT INTO Is_at_Locations VALUES'
    for place in data:
        loc = place['loc']
        value = (place['rid'],loc['stnum'],loc['stname'],loc['city'], loc['state'],loc['postal'])
        print(base_insert,value,';')

In [157]:
insert_is_at_location(result)

INSERT INTO Is_at_Locations VALUES ('ChIJu3iXyGr2wokR29wWAqLfn8Q', '1280', 'Amsterdam Ave', 'New York', 'NY', '10027') ;
INSERT INTO Is_at_Locations VALUES ('ChIJUZGzuBL3wokRslOX3yc9LCk', '998', 'Columbus Ave', 'New York', 'NY', '10025') ;
INSERT INTO Is_at_Locations VALUES ('ChIJSQm0BGr2wokRMlxPSzeSl2g', '3183', 'Broadway', 'New York', 'NY', '10027') ;
INSERT INTO Is_at_Locations VALUES ('ChIJD86ws4JYwokRz3jXJAWgGdM', '685', 'Amsterdam Ave', 'New York', 'NY', '10025') ;
INSERT INTO Is_at_Locations VALUES ('ChIJjYXMA6r3wokRqsATKxPO-Vg', 'STE', '350', 'New York', 'NY', '10027') ;
INSERT INTO Is_at_Locations VALUES ('ChIJVVUpOG_2wokRFUtuo_v80Ts', '1504', 'Amsterdam Ave', 'New York', 'NY', '10031') ;
INSERT INTO Is_at_Locations VALUES ('ChIJFalicmT2wokR02HksZFX9VA', '3470', 'Broadway', 'New York', 'NY', '10031') ;
INSERT INTO Is_at_Locations VALUES ('ChIJYbl1lBL2wokR3Pw0NejRRvM', '2071', 'Adam Clayton Powell Jr Blvd', 'New York', 'NY', '10027') ;
INSERT INTO Is_at_Locations VALUES ('ChIJg

In [125]:
def insert_user_reviews(data):
    base_insert = 'INSERT INTO User_Reviews VALUES'
    for place in data:
        rev = place['rev']
        if rev == None:
            continue
        else:
            for review in rev:
                urid = review['urid']
                rating = review['rating']
                detail = review['detail']
                value = (urid,rating,detail)
                print(base_insert,value,';')

In [158]:
insert_user_reviews(result)

INSERT INTO User_Reviews VALUES ('1073257240094137136122', 5, 'This restaurant is no doubt one of the tastiest place to eat in the neighborhood. Always when I go there I am incredibly happy. They manage their high  level service and the highest level of food  they offer. You will be always satisfied with this restaurant. Very  recommended.') ;
INSERT INTO User_Reviews VALUES ('1111309857000646600308', 5, "Haven't visited since covid took over. But hands down top 3 in all categories. Food, drinks but the best is ambience!🔥 Wont mention any spots that might have a rep but ill take Nikkos any day over a franchise.") ;
INSERT INTO User_Reviews VALUES ('1116762536143342823813', 5, 'This place is awesome! Good prices, big dishes and the staff are always very nice.') ;
INSERT INTO User_Reviews VALUES ('1161273201578486632477', 5, 'Amazing staff, who are not only very professional, but also super helpful. Highly recommend the calamari.') ;
INSERT INTO User_Reviews VALUES ('10061116072738224052

In [115]:
def insert_user_writes_for(data):
    base_insert = 'INSERT INTO u_writes_for VALUES'
    for place in data:
        rid = place['rid']
        rev = place['rev']
        if rev == None:
            continue
        else:
            for review in rev:
                uid = review['uid']
                urid = review['urid']
                time = review['time']
                value = (uid,urid,rid,time)
                print(base_insert,value,';')

In [159]:
insert_user_writes_for(result)

INSERT INTO u_writes_for VALUES ('107325724009413713612', '1073257240094137136122', 'ChIJu3iXyGr2wokR29wWAqLfn8Q', '2020-09-21 18:59:19') ;
INSERT INTO u_writes_for VALUES ('111130985700064660030', '1111309857000646600308', 'ChIJu3iXyGr2wokR29wWAqLfn8Q', '2020-10-18 03:49:40') ;
INSERT INTO u_writes_for VALUES ('111676253614334282381', '1116762536143342823813', 'ChIJu3iXyGr2wokR29wWAqLfn8Q', '2020-09-28 22:16:09') ;
INSERT INTO u_writes_for VALUES ('116127320157848663247', '1161273201578486632477', 'ChIJu3iXyGr2wokR29wWAqLfn8Q', '2020-08-20 03:50:42') ;
INSERT INTO u_writes_for VALUES ('100611160727382240523', '1006111607273822405232', 'ChIJu3iXyGr2wokR29wWAqLfn8Q', '2020-07-21 13:06:51') ;
INSERT INTO u_writes_for VALUES ('113940049200647362048', '1139400492006473620487', 'ChIJUZGzuBL3wokRslOX3yc9LCk', '2020-10-17 01:46:51') ;
INSERT INTO u_writes_for VALUES ('113350429450063061606', '1133504294500630616062', 'ChIJUZGzuBL3wokRslOX3yc9LCk', '2020-10-17 22:39:44') ;
INSERT INTO u_writes

In [66]:
def insert_provides_menu(data):
    base_insert = 'INSERT INTO Provides_Menu VALUES'
    for place in data:
        value = (place['mid'],place['rid'])
        print(base_insert,value,';')

In [160]:
insert_provides_menu(result)

INSERT INTO Provides_Menu VALUES (1, 'ChIJu3iXyGr2wokR29wWAqLfn8Q') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJUZGzuBL3wokRslOX3yc9LCk') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJSQm0BGr2wokRMlxPSzeSl2g') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJD86ws4JYwokRz3jXJAWgGdM') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJjYXMA6r3wokRqsATKxPO-Vg') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJVVUpOG_2wokRFUtuo_v80Ts') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJFalicmT2wokR02HksZFX9VA') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJYbl1lBL2wokR3Pw0NejRRvM') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJg0Bl54JYwokR8k8qF8pW-eI') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJU-oKKh1ZwokR9HVy197A8mM') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJEY8Sdlf2wokRCnHpToa-BE0') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJfebAhnn3wokRvTbzHU2Scwg') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJgTrKAYZYwokRrLF7JiGwdNM') ;
INSERT INTO Provides_Menu VALUES (1, 'ChIJ1S4Bt45YwokRrLS1-axV2_Q') ;
INSERT INTO Provides

In [22]:
def insert_satisfies(data):
    base_insert = 'INSERT INTO Satisfies VALUES'
    for place in data:
        for name in place['dname']:
            value = (name, place['rid'],place['mid'])
            print(base_insert,value,';')

In [162]:
insert_satisfies(result)

INSERT INTO Satisfies VALUES ('Vegan', 'ChIJu3iXyGr2wokR29wWAqLfn8Q', 1) ;
INSERT INTO Satisfies VALUES ('Vegetarian', 'ChIJu3iXyGr2wokR29wWAqLfn8Q', 1) ;
INSERT INTO Satisfies VALUES ('Vegetarian', 'ChIJUZGzuBL3wokRslOX3yc9LCk', 1) ;
INSERT INTO Satisfies VALUES ('Gluten Free', 'ChIJUZGzuBL3wokRslOX3yc9LCk', 1) ;
INSERT INTO Satisfies VALUES ('Vegan', 'ChIJSQm0BGr2wokRMlxPSzeSl2g', 1) ;
INSERT INTO Satisfies VALUES ('Vegetarian', 'ChIJSQm0BGr2wokRMlxPSzeSl2g', 1) ;
INSERT INTO Satisfies VALUES ('Vegan', 'ChIJD86ws4JYwokRz3jXJAWgGdM', 1) ;
INSERT INTO Satisfies VALUES ('Vegetarian', 'ChIJD86ws4JYwokRz3jXJAWgGdM', 1) ;
INSERT INTO Satisfies VALUES ('Halal', 'ChIJD86ws4JYwokRz3jXJAWgGdM', 1) ;
INSERT INTO Satisfies VALUES ('Vegan', 'ChIJjYXMA6r3wokRqsATKxPO-Vg', 1) ;
INSERT INTO Satisfies VALUES ('Vegetarian', 'ChIJjYXMA6r3wokRqsATKxPO-Vg', 1) ;
INSERT INTO Satisfies VALUES ('Vegetarian', 'ChIJVVUpOG_2wokRFUtuo_v80Ts', 1) ;
INSERT INTO Satisfies VALUES ('Gluten Free', 'ChIJVVUpOG_2wokRFU